In [ ]:
import sys
from pathlib import Path
import pandas as pd

# Add the src folder to Python's module search path
sys.path.append(str(Path("../src").resolve()))

from ingestion.ingestion import load_all_data

## 1. Data Loading

In [55]:
dfs = load_all_data()

## 2. Data Inspection

All identified issues are logged in "data/logs/data_issues.csv"

### 2.1 Customer Data

In [27]:
import pandas as pd

def basic_eda(df: pd.DataFrame, name: str = "Dataset") -> pd.DataFrame:
    """
    Perform basic exploratory data analysis on a DataFrame.
    
    Prints summary stats and returns a summary DataFrame for further use.
    
    Args:
        df (pd.DataFrame): Input DataFrame
        name (str): Name of the dataset (for printing)
        
    Returns:
        pd.DataFrame: Summary table with column info
    """
    print(f"\n=== EDA for {name} ===\n")
    
    # Basic shape info
    print(f"Rows: {df.shape[0]}, Columns: {df.shape[1]}\n")
    
    # Missing values
    missing = df.isnull().sum()
    print("Missing values per column:")
    print(missing[missing > 0])
    
    # Duplicates
    dup_count = df.duplicated().sum()
    print(f"\nDuplicate rows: {dup_count}")
    
    # Data types
    print("\nData types:")
    print(df.dtypes)
    
    # Basic statistics for numerical columns
    print("\nNumerical stats:")
    print(df.describe())
    
    # Basic statistics for categorical columns
    cat_cols = df.select_dtypes(include='object').columns
    if len(cat_cols) > 0:
        print("\nCategorical value counts (top 5 per column):")
        for col in cat_cols:
            print(f"\n{col}:")
            print(df[col].value_counts(dropna=False).head(5))
    
    # Unique values per column
    unique_counts = df.nunique()
    
    # Create summary DataFrame for logging / inspection
    summary_df = pd.DataFrame({
        "column": df.columns,
        "dtype": df.dtypes,
        "n_unique": df.nunique(),
        "n_missing": df.isnull().sum(),
        "n_duplicates": [df.duplicated(subset=[c]).sum() for c in df.columns]
    })
    
    return summary_df

In [29]:
basic_eda(dfs["customers"], "customers")


=== EDA for customers ===

Rows: 1520, Columns: 7

Missing values per column:
country       81
age           74
gender       305
plan_type    310
dtype: int64

Duplicate rows: 20

Data types:
customer_id     object
signup_date     object
country         object
age            float64
gender          object
plan_type       object
monthly_fee    float64
dtype: object

Numerical stats:
               age  monthly_fee
count  1446.000000  1520.000000
mean     49.740664    32.187987
std      17.940998    15.683184
min      18.000000     5.010000
25%      34.000000    18.810000
50%      50.000000    31.890000
75%      66.000000    45.692500
max      79.000000    59.960000

Categorical value counts (top 5 per column):

customer_id:
customer_id
C100249    2
C100369    2
C100707    2
C100383    2
C100002    2
Name: count, dtype: int64

signup_date:
signup_date
2023-08-15    4
2023-04-29    4
2021-08-15    4
2021-09-19    4
2021-11-28    4
Name: count, dtype: int64

country:
country
ES    312
IT 

,column,dtype,n_unique,n_missing,n_duplicates
customer_id,customer_id,object,1500,0,20
signup_date,signup_date,object,1189,0,331
country,country,object,6,81,1513
age,age,float64,62,74,1457
gender,gender,object,4,305,1515
plan_type,plan_type,object,4,310,1515
monthly_fee,monthly_fee,float64,1312,0,208


**Verdict:**
- Remove duplicates for customer IDs
- Transform signup data to date type
- Gender and plan type with > 30% missing / dirty data. Consider dropping columns as number to big

In [30]:
basic_eda(dfs["transactions"], "transactions")


=== EDA for transactions ===

Rows: 28677, Columns: 6

Missing values per column:
Series([], dtype: int64)

Duplicate rows: 0

Data types:
customer_id       object
date              object
call_minutes     float64
data_usage_gb    float64
sms_count          int64
amount_paid      float64
dtype: object

Numerical stats:
       call_minutes  data_usage_gb     sms_count   amount_paid
count  28677.000000   28677.000000  28677.000000  28677.000000
mean     299.905934       4.436440     49.438295     57.351074
std       99.952097       2.619344     20.039861     35.977324
min     -147.852229      -5.147895      0.000000     -4.999715
25%      232.208232       3.116473     36.000000     26.252761
50%      299.168579       4.752098     49.000000     57.293625
75%      368.358870       6.193105     63.000000     88.329442
max      695.511493      12.111234    143.000000    119.988836

Categorical value counts (top 5 per column):

customer_id:
customer_id
C999999    60
C100468    35
C100636    

,column,dtype,n_unique,n_missing,n_duplicates
customer_id,customer_id,object,1501,0,27176
date,date,object,35,0,28642
call_minutes,call_minutes,float64,28677,0,0
data_usage_gb,data_usage_gb,float64,28677,0,0
sms_count,sms_count,int64,125,0,28552
amount_paid,amount_paid,float64,28677,0,0


**Verdict:**
- Negative values in call_minutes, data_usage_gb and sms_count to be removed
- Transform signup data to date type
- some transactions with undefined customer id (C999999)

In [ ]:
dfs["transactions"][dfs["transactions"].customer_id == "C100000"]
# transaction date sometimes falls in the previous month

,customer_id,date,call_minutes,data_usage_gb,sms_count,amount_paid
0,C100000,2021-01-01,257.036502,2.397781,41,76.827649
1,C100000,2021-01-31,396.476989,6.355708,82,39.655246
2,C100000,2021-03-02,116.148138,6.796235,42,76.287122
3,C100000,2021-04-01,370.253331,2.962127,17,27.764122
4,C100000,2021-05-01,200.190537,1.491430,51,112.945652
5,C100000,2021-05-31,480.608912,5.007923,38,68.028099
6,C100000,2021-06-30,411.668128,-1.232095,59,-2.619574
7,C100000,2021-07-30,265.785135,5.460111,52,77.149708
8,C100000,2021-08-29,242.154027,4.845011,48,-1.375407
9,C100000,2021-09-28,382.640332,4.625933,89,10.543165


In [34]:
basic_eda(dfs["support_interactions"], "support")


=== EDA for support ===

Rows: 2500, Columns: 7

Missing values per column:
customer_id              3
channel                491
issue_type             525
resolution_time_min    201
was_resolved           875
dtype: int64

Duplicate rows: 0

Data types:
ticket_id               object
customer_id             object
timestamp               object
channel                 object
issue_type              object
resolution_time_min    float64
was_resolved           float64
dtype: object

Numerical stats:
       resolution_time_min  was_resolved
count          2299.000000   1625.000000
mean            149.478034      0.462769
std              85.674256      0.498765
min               1.000000      0.000000
25%              75.000000      0.000000
50%             149.000000      0.000000
75%             222.000000      1.000000
max             299.000000      1.000000

Categorical value counts (top 5 per column):

ticket_id:
ticket_id
T51386    2
T28534    2
T67367    2
T23814    2
T31034   

,column,dtype,n_unique,n_missing,n_duplicates
ticket_id,ticket_id,object,2473,0,27
customer_id,customer_id,object,1199,3,1300
timestamp,timestamp,object,1657,0,843
channel,channel,object,4,491,2495
issue_type,issue_type,object,4,525,2495
resolution_time_min,resolution_time_min,float64,299,201,2200
was_resolved,was_resolved,float64,2,875,2497


**Verdict:**
- Transform timestamp into date format
- many missing values in channel, issue_type and was resolved (>30%). Will most likely have to default on aggregated features on interactions without details

In [36]:
dfs["support_interactions"].head()

,ticket_id,customer_id,timestamp,channel,issue_type,resolution_time_min,was_resolved
0,T67343,C101163,2021-10-15 00:00:00,email,technical,139.0,1.0
1,T40570,C100442,16/08/2023 00:00,chat,billing,272.0,NaN
2,T39048,C101154,03/10/2023 00:00,NaN,technical,234.0,0.0
3,T57182,C101340,2023-11-07 00:00:00,email,billing,30.0,NaN
4,T32047,C101040,2023-11-26 00:00:00,phone,billing,150.0,0.0
